In [1]:
# ------------------------------------------------------------------------------
#
#  GMSH-CAD Groningen model desgin 
#
#  STEP import and manipulation, geometry partitioning
#
# ------------------------------------------------------------------------------

In [2]:
import gmsh
import math
import os
import sys
import numpy as np

In [3]:
def readfault(file,xmin=0,ymin=0,zmin=0):
    ps = []
    with codecs.open(file, encoding='utf-8-sig') as f:
        X = np.loadtxt(f)
    X[:,0] = X[:,0] - xmin
    X[:,1] = X[:,1] - ymin
    X[:,2] = X[:,2] - zmin
    for x in X[:]:
        ps.append(gmsh.model.occ.addPoint(float(x[0]), float(x[1]), float(x[2])))
    return ps

In [4]:
gmsh.initialize()
gmsh.model.add("spline")

In [5]:
import numpy as np
import codecs

with codecs.open('f1.txt', encoding='utf-8-sig') as f:
    X = np.loadtxt(f)
    
xmin, ymin, zmin, xmax, ymax, zmax = np.min(X[:,0]),np.min(X[:,1]),np.min(X[:,2]),np.max(X[:,0]),np.max(X[:,1]),np.max(X[:,1])


In [6]:
# read fault
ps1 = readfault("faults/M1.txt")
ps2 = readfault("faults/mFS7_Fault_52.txt")
ps3 = readfault("faults/mFS7_Fault_53.txt") # down
# ps3 = readfault("faults/f53_meger_LSSB.txt")
ps4 = readfault("faults/f54_1.txt") # left
ps5 = readfault("faults/f54_2.txt") # right
ps6 = readfault("faults/f38_1.txt")
ps7 = readfault("faults/f38_2.txt")
ps8 = readfault("faults/f38_3.txt")
ps9 = readfault("faults/fm69_1.txt")
ps10 = readfault("faults/fm69_2.txt")
ps11 = readfault("faults/fm69_3.txt")

ps12 = readfault("faults/f54_ex.txt")
ps13 = readfault("faults/f54_ex2.txt")
ps14 = readfault("faults/extend_f54_2_xflt.txt")

f1 = gmsh.model.occ.addBSplineSurface(ps1, numPointsU=len(ps1)//10)
f2 = gmsh.model.occ.addBSplineSurface(ps2, numPointsU=len(ps2)//10)
f3 = gmsh.model.occ.addBSplineSurface(ps3, numPointsU=len(ps3)//10)
f4 = gmsh.model.occ.addBSplineSurface(ps4, numPointsU=len(ps4)//10)
f5 = gmsh.model.occ.addBSplineSurface(ps5, numPointsU=len(ps5)//10)
f6 = gmsh.model.occ.addBSplineSurface(ps6, numPointsU=len(ps6)//10)
f7 = gmsh.model.occ.addBSplineSurface(ps7, numPointsU=len(ps7)//10)
f8 = gmsh.model.occ.addBSplineSurface(ps8, numPointsU=len(ps8)//10)
f9 = gmsh.model.occ.addBSplineSurface(ps9, numPointsU=len(ps9)//10)
f10 = gmsh.model.occ.addBSplineSurface(ps10, numPointsU=len(ps10)//10)
f11 = gmsh.model.occ.addBSplineSurface(ps11, numPointsU=len(ps11)//10)
f12 = gmsh.model.occ.addBSplineSurface(ps12, numPointsU=len(ps12)//10)
f13 = gmsh.model.occ.addBSplineSurface(ps13, numPointsU=len(ps13)//10)
f14 = gmsh.model.occ.addBSplineSurface(ps14, numPointsU=len(ps14)//10)


gmsh.model.occ.synchronize()
gmsh.model.occ.removeAllDuplicates()

In [7]:
# manually
box = gmsh.model.occ.addBox(245317-1000+180, 597601-1000-180, -2350, 2000, 2000, -1000)
gmsh.model.occ.rotate([(3,box)], x=245317+180, y=597601-180, z=-5350, ax=0, ay=0, az=4000, angle=-30/180*np.pi)

gmsh.model.occ.fragment(gmsh.model.occ.getEntities(3),
                            gmsh.model.occ.getEntities(2))
gmsh.model.occ.synchronize()


Info    : Recomputing incorrect OpenCASCADE wire in surface 69


In [8]:
# zero-offset modelling
s3 = gmsh.model.occ.importShapes('surfaces/LSSB3.step')
s4 = gmsh.model.occ.importShapes('surfaces/LSSB4.step')
t3 = gmsh.model.occ.importShapes('surfaces/ROT3.step')
t4 = gmsh.model.occ.importShapes('surfaces/ROT4.step')
gmsh.model.occ.translate(s4, -0.20199935999698937, -0.15324089373461902, 6.754887411052096) # manually
gmsh.model.occ.translate(t4, -0.5888777230575215, -0.4467348244506866, 19.69216558035623) # manually

at3 = gmsh.model.occ.importShapes('surfaces/ROT3.step')
at4 = gmsh.model.occ.importShapes('surfaces/ROT4.step')
gmsh.model.occ.translate(at4, -0.5888777230575215-0.00293468, -0.4467348244506866-0.00222631, 69.69216558035623+0.09813601) # manually
gmsh.model.occ.translate(at3,0, 0, 50)
# gmsh.model.getValue(0,6041,[])-gmsh.model.getValue(0,6009,[])

gmsh.model.occ.synchronize()
gmsh.model.occ.fragment([(3,3)], [s3[0],t3[0],at3[0]],removeObject=True, removeTool=True)
gmsh.model.occ.fragment([(3,5)], [s4[0],t4[0],at4[0]],removeObject=True, removeTool=True)
# gmsh.model.occ.remove(s3, recursive=True)
# gmsh.model.occ.remove(s4, recursive=True)
gmsh.model.occ.remove(gmsh.model.occ.getEntities(2),recursive=True)
gmsh.model.occ.remove(gmsh.model.occ.getEntities(1),recursive=True)
gmsh.model.occ.remove(gmsh.model.occ.getEntities(0),recursive=True)

# np1 = gmsh.model.occ.addPoint(246317, 596601, -3143) # manually
# np2 = gmsh.model.occ.addPoint(246317, 596601, -2866) # manually
# np3 = gmsh.model.occ.addPoint(246317, 596601, -2866+50) # manually

gmsh.model.occ.synchronize()
gmsh.model.occ.removeAllDuplicates()



Info    :  - Label 'Shapes/Surface001' (2D)
Info    :  - Label 'Shapes/Surface002' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)


In [9]:
# manually setting a extra zone for zero-offset tip

# exl1 = gmsh.model.occ.addLine(6051,np1)
# exl2 = gmsh.model.occ.addLine(np1,6056)
# exl3 = gmsh.model.occ.addLine(6053,np2)
# exl4 = gmsh.model.occ.addLine(np2,6057)

exl1 = gmsh.model.occ.addLine(6104,6109)
exl2 = gmsh.model.occ.addLine(6102,6108)
exl3 = gmsh.model.occ.addLine(6100,6107)



gmsh.model.occ.addCurveLoop([338,exl1,-350], 1)
s111 = gmsh.model.occ.addSurfaceFilling(1, tag=-1) 
gmsh.model.occ.addCurveLoop([335,exl2,-349], 2)
s222 = gmsh.model.occ.addSurfaceFilling(2, tag=-1) 
gmsh.model.occ.addCurveLoop([332,exl3,-348], 3)
s333 = gmsh.model.occ.addSurfaceFilling(3, tag=-1) 

gmsh.model.occ.fragment([(3,6)], [(2,s111),(2,s222),(2,s333)],removeObject=True, removeTool=True)
gmsh.model.occ.synchronize()

Info    : Recomputing incorrect OpenCASCADE wire in surface 144
Info    : Recomputing incorrect OpenCASCADE wire in surface 147
Info    : Recomputing incorrect OpenCASCADE wire in surface 148
Info    : Recomputing incorrect OpenCASCADE wire in surface 158
Info    : Recomputing incorrect OpenCASCADE wire in surface 172
Info    : Recomputing incorrect OpenCASCADE wire in surface 176
Info    : Recomputing incorrect OpenCASCADE wire in surface 196
Info    : Recomputing incorrect OpenCASCADE wire in surface 198
Info    : Recomputing incorrect OpenCASCADE wire in surface 201
Info    : Recomputing incorrect OpenCASCADE wire in surface 203
Info    : Recomputing incorrect OpenCASCADE wire in surface 206
Info    : Recomputing incorrect OpenCASCADE wire in surface 208
Info    : Recomputing incorrect OpenCASCADE wire in surface 212


In [10]:
# horizon modelling
s1 = gmsh.model.occ.importShapes('surfaces/LSSB1.step')
s2 = gmsh.model.occ.importShapes('surfaces/LSSB2.step')
s5 = gmsh.model.occ.importShapes('surfaces/LSSB5.step')
s6 = gmsh.model.occ.importShapes('surfaces/LSSB6.step')
t1 = gmsh.model.occ.importShapes('surfaces/ROT1.step')
t2 = gmsh.model.occ.importShapes('surfaces/ROT2.step')
t5 = gmsh.model.occ.importShapes('surfaces/ROT5.step')
t6 = gmsh.model.occ.importShapes('surfaces/ROT6.step')

# anhydrite
at1 = gmsh.model.occ.importShapes('surfaces/ROT1.step')
at2 = gmsh.model.occ.importShapes('surfaces/ROT2.step')
at5 = gmsh.model.occ.importShapes('surfaces/ROT5.step')
at6 = gmsh.model.occ.importShapes('surfaces/ROT6.step')
gmsh.model.occ.translate(at1,0, 0, 50)
gmsh.model.occ.translate(at2,0, 0, 50)
gmsh.model.occ.translate(at5,0, 0, 50)
gmsh.model.occ.translate(at6,0, 0, 50)


Info    :  - Label 'Shapes/Surface003' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)
Info    :  - Label 'Shapes/Surface011' (2D)


In [11]:
gmsh.model.occ.fragment([(3,7)], [s1[0],t1[0],at1[0]],removeObject=True, removeTool=True)
gmsh.model.occ.fragment([(3,2)], [s2[0],t2[0],at2[0]],removeObject=True, removeTool=True)
gmsh.model.occ.fragment([(3,1)], [s5[0],t5[0],at5[0]],removeObject=True, removeTool=True)
gmsh.model.occ.fragment([(3,4)], [s6[0],t6[0],at6[0]],removeObject=True, removeTool=True)
# gmsh.model.occ.remove(s1, recursive=True)
# gmsh.model.occ.remove(s2, recursive=True)
# gmsh.model.occ.remove(s5, recursive=True)
# gmsh.model.occ.remove(s6, recursive=True)
gmsh.model.occ.remove(gmsh.model.occ.getEntities(2),recursive=True)
gmsh.model.occ.remove(gmsh.model.occ.getEntities(1),recursive=True)
gmsh.model.occ.remove(gmsh.model.occ.getEntities(0),recursive=True)
gmsh.model.occ.removeAllDuplicates()
gmsh.model.occ.synchronize()

Info    : Recomputing incorrect OpenCASCADE wire in surface 356
Info    : Recomputing incorrect OpenCASCADE wire in surface 358
Info    : Recomputing incorrect OpenCASCADE wire in surface 362
Info    : Recomputing incorrect OpenCASCADE wire in surface 366
Info    : Recomputing incorrect OpenCASCADE wire in surface 375
Info    : Recomputing incorrect OpenCASCADE wire in surface 378
Info    : Recomputing incorrect OpenCASCADE wire in surface 381
Info    : Recomputing incorrect OpenCASCADE wire in surface 382
Info    : Recomputing incorrect OpenCASCADE wire in surface 385


In [12]:
# Manual
# point 6246,6245,6165,6231
cp1 = (gmsh.model.getValue(0,6279,[])+gmsh.model.getValue(0,6277,[]))/2
cp2 = (gmsh.model.getValue(0,6277,[])+gmsh.model.getValue(0,6197,[]))/2
cp3 = (gmsh.model.getValue(0,6197,[])+gmsh.model.getValue(0,6263,[]))/2
cp4 = (gmsh.model.getValue(0,6263,[])+gmsh.model.getValue(0,6279,[]))/2

In [13]:
tp5 = gmsh.model.occ.addPoint(cp1[0],cp1[1],cp1[2])
tp6 = gmsh.model.occ.addPoint(cp2[0],cp2[1],cp2[2])
tp7 = gmsh.model.occ.addPoint(cp3[0],cp3[1],cp3[2])
tp8 = gmsh.model.occ.addPoint(cp4[0],cp4[1],cp4[2])

gmsh.model.occ.fragment(gmsh.model.occ.getEntities(3),[(0,tp5),(0,tp6),(0,tp7),(0,tp8)])
gmsh.model.occ.synchronize()

Info    : Recomputing incorrect OpenCASCADE wire in surface 397
Info    : Recomputing incorrect OpenCASCADE wire in surface 398
Info    : Recomputing incorrect OpenCASCADE wire in surface 400
Info    : Recomputing incorrect OpenCASCADE wire in surface 401
Info    : Recomputing incorrect OpenCASCADE wire in surface 407
Info    : Recomputing incorrect OpenCASCADE wire in surface 408


In [14]:
gmsh.model.occ.synchronize()

for i in [28,24,32,8,16,12,23]:
    tag = gmsh.model.addPhysicalGroup(3, [i], tag=-1)
    gmsh.model.setPhysicalName(3, tag, ("overburden"+str(i)))
for i in [29,25,33,9,17,13,22]:
    tag = gmsh.model.addPhysicalGroup(3, [i], tag=-1)
    gmsh.model.setPhysicalName(3, tag, ("seal"+str(i)))
for i in [30,26,34,10,18,14,20]:
    tag = gmsh.model.addPhysicalGroup(3, [i], tag=-1)
    gmsh.model.setPhysicalName(3, tag, ("reservoir"+str(i)))
for i in [31,27,35,11,15,19,21]:
    tag = gmsh.model.addPhysicalGroup(3, [i], tag=-1)
    gmsh.model.setPhysicalName(3, tag, ("underburden"+str(i)))
    


In [15]:
# physical surface(model boundary/ fault) 
phy_s1 = gmsh.model.addPhysicalGroup(2, [394,386,220,351,279,210,243], tag=-1)
phy_s2 = gmsh.model.addPhysicalGroup(2, [315,407,216,333,405,397,403], tag=-1)
phy_s3 = gmsh.model.addPhysicalGroup(2, [396,404,406,359,263,377,234,363,370,381,241,209,277,385], tag=-1)
phy_s4 = gmsh.model.addPhysicalGroup(2, [318,335,217,399,402,390,344,218,361,247,392,349,219,365,238,395,353,221,211,245], tag=-1)
phy_s5 = gmsh.model.addPhysicalGroup(2, [408,319,378,389,382,391,387,393], tag=-1)
phy_s6 = gmsh.model.addPhysicalGroup(2, [401,398,246,360,235,364,242], tag=-1)
gmsh.model.setPhysicalName(2, phy_s1, "bot")
gmsh.model.setPhysicalName(2, phy_s2, "top")
gmsh.model.setPhysicalName(2, phy_s3, "-x")
gmsh.model.setPhysicalName(2, phy_s4, "+x")
gmsh.model.setPhysicalName(2, phy_s5, "-y")
gmsh.model.setPhysicalName(2, phy_s6, "+y")

In [16]:
# fault segment based on the intersection location.
for i in [259,264,265,271,272,278,141]: # top down, west east (fault plane) (x1)
    fault_grp = gmsh.model.addPhysicalGroup(2, [i], tag=-1)
    gmsh.model.setPhysicalName(2, fault_grp, ("flt1M-"+str(i)))

for i in [178,342,182,347,186,352,190]: # top down, west east (fault plane) (x1)
    fault_grp = gmsh.model.addPhysicalGroup(2, [i], tag=-1)
    gmsh.model.setPhysicalName(2, fault_grp, ("flt1A-"+str(i)))

for i in [334,340,268,269,345,276,281]: # top down, west east (fault plane) (x1) [6,169,142,141,173,146,147,111]
    fault_grp = gmsh.model.addPhysicalGroup(2, [i], tag=-1)
    gmsh.model.setPhysicalName(2, fault_grp, ("flt2M-"+str(i)))

for i in [400,362,366,212]: # top down, west east (fault plane) (x1) [6,169,142,141,173,146,147,111]
    fault_grp = gmsh.model.addPhysicalGroup(2, [i], tag=-1)
    gmsh.model.setPhysicalName(2, fault_grp, ("flt2A-"+str(i)))
    


In [17]:
# physical line(fault boundary)
phy_l1 = gmsh.model.addPhysicalGroup(1, [311,358,341,339,336,333,368,68,291,539,529,528,518,517,509,\
                                        643,562,573,572,651,527,526,654,595,537,543,319,394,395,668,666,699,698], tag=-1)
gmsh.model.setPhysicalName(1, phy_l1, "FB")

# intersection
phy_l1 = gmsh.model.addPhysicalGroup(1, [185,513,521,316,523,532,318,534,542,294], tag=-1)
gmsh.model.setPhysicalName(1, phy_l1, "FX")

In [18]:
# mesh size field (adaptive mesh size based on the distance from the fault and the reservoir boundaries.)
lc = 400
# mesh size near the fault
gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "FacesList", [264,265,271,272,278,342,182,347,186,352,340,268,269,345,276])
gmsh.model.mesh.field.setNumber(1, "Sampling", 2000)
gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "InField", 1)
gmsh.model.mesh.field.setNumber(2, "SizeMin", 2) # 6
gmsh.model.mesh.field.setNumber(2, "SizeMax", 100)
gmsh.model.mesh.field.setNumber(2, "DistMin", 4.0) # 12
gmsh.model.mesh.field.setNumber(2, "DistMax", 12.0)

# # reservoir interval
# gmsh.model.mesh.field.add("Box", 3)
# gmsh.model.mesh.field.setNumber(3, "VIn", 20)
# gmsh.model.mesh.field.setNumber(3, "VOut", 200)
# gmsh.model.mesh.field.setNumber(3, "XMin", 2.4e5)
# gmsh.model.mesh.field.setNumber(3, "XMax", 2.5e5)
# gmsh.model.mesh.field.setNumber(3, "YMin", 5.95e5)
# gmsh.model.mesh.field.setNumber(3, "YMax", 6.0e5)
# gmsh.model.mesh.field.setNumber(3, "ZMin", -3200)
# gmsh.model.mesh.field.setNumber(3, "ZMax", -2760)
# gmsh.model.mesh.field.setNumber(6, "Thickness", 0.3)

gmsh.model.mesh.field.add("Distance", 4)
gmsh.model.mesh.field.setNumbers(4, "FacesList", [294,301,323,329,273,266,185,181,214,215,201,206,239,236])
gmsh.model.mesh.field.setNumber(4, "Sampling", 2000)
gmsh.model.mesh.field.add("Threshold", 5)
gmsh.model.mesh.field.setNumber(5, "InField", 4)
gmsh.model.mesh.field.setNumber(5, "SizeMin", 30)
gmsh.model.mesh.field.setNumber(5, "SizeMax", 100)
gmsh.model.mesh.field.setNumber(5, "DistMin", 1.0)
gmsh.model.mesh.field.setNumber(5, "DistMax", 600.0)

gmsh.model.mesh.field.add("Min", 7)
gmsh.model.mesh.field.setNumbers(7, "FieldsList", [2,5])

gmsh.model.mesh.field.setAsBackgroundMesh(7)

gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromPoints", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 0)

# Finally, while the default "Frontal-Delaunay" 2D meshing algorithm
# (Mesh.Algorithm = 6) usually leads to the highest quality meshes, the
# "Delaunay" algorithm (Mesh.Algorithm = 5) will handle complex mesh size fields
# better - in particular size fields with large element size gradients:

gmsh.option.setNumber("Mesh.Algorithm", 6)

gmsh.model.mesh.generate(3)
gmsh.write("Zeerijp_anhydrite_xflt.msh")


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 68 (BSpline)
Info    : [ 10%] Meshing curve 73 (BSpline)
Info    : [ 10%] Meshing curve 80 (BSpline)
Info    : [ 10%] Meshing curve 91 (BSpline)
Info    : [ 10%] Meshing curve 97 (BSpline)
Info    : [ 10%] Meshing curve 185 (BSpline)
Info    : [ 10%] Meshing curve 194 (BSpline)
Info    : [ 10%] Meshing curve 198 (BSpline)
Info    : [ 10%] Meshing curve 200 (BSpline)
Info    : [ 10%] Meshing curve 210 (BSpline)
Info    : [ 10%] Meshing curve 212 (BSpline)
Info    : [ 10%] Meshing curve 221 (BSpline)
Info    : [ 10%] Meshing curve 224 (BSpline)
Info    : [ 10%] Meshing curve 250 (BSpline)
Info    : [ 10%] Meshing curve 261 (BSpline)
Info    : [ 10%] Meshing curve 266 (BSpline)
Info    : [ 10%] Meshing curve 271 (BSpline)
Info    : [ 10%] Meshing curve 277 (BSpline)
Info    : [ 10%] Meshing curve 279 (BSpline)
Info    : [ 10%] Meshing curve 280 (BSpline)
Info    : [ 10%] Meshing curve 289 (BSpline)
Info    : [ 10%] Meshing curve 291 (

In [ ]:
gmsh.model.occ.synchronize()

if '-nopopup' not in sys.argv:
    gmsh.fltk.run()
gmsh.finalize()

-------------------------------------------------------
Version       : 4.9.0
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20211203
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.0
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------
